# Data Fetching 
in this notebook data will be fetching using *Post* requests to specific api 

Importing Packages

In [1]:
import pandas as pd 
import requests
import json
import math
URL = "https://recruitment.aimtechnologies.co/ai-tasks"

In [2]:
df = pd.read_csv('dialect_dataset.csv')
df.head()

,id,dialect
0,1175358310087892992,IQ
1,1175416117793349632,IQ
2,1175450108898565888,IQ
3,1175471073770573824,IQ
4,1175496913145217024,IQ


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458197 entries, 0 to 458196
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       458197 non-null  int64 
 1   dialect  458197 non-null  object
dtypes: int64(1), object(1)
memory usage: 7.0+ MB


In [5]:
df.isna().sum()

id         0
dialect    0
dtype: int64

The df has no missing values 

In [4]:
df['id'] = df['id'].apply(str) # convert id from object to string to be able to send it in requests
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458197 entries, 0 to 458196
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       458197 non-null  object
 1   dialect  458197 non-null  object
dtypes: object(2)
memory usage: 7.0+ MB


In [7]:
Tweets_json = '{}'
Tweets_json = json.loads(Tweets_json)
for i in range (0, math.ceil(len(df)/1000)): # generete Lists of 1000 ids
    max = 999
    if(i*1000 + max > len(df)):
        max = len(df) - i*1000 
        
    ids =df.loc[i*1000:i*1000+max]['id'].values.tolist()   
    ids_json = json.dumps(ids)

    r = requests.post(URL ,ids_json)
    data = r.json()
    Tweets_json.update(data)

In [8]:
tweets_df = pd.DataFrame.from_dict(Tweets_json,orient="index")
tweets_df.head()

,0
1175358310087892992,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1175416117793349632,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
1175450108898565888,@KanaanRema مبين من كلامه خليجي
1175471073770573824,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
1175496913145217024,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺


In [9]:
tweets_df.columns= ['tweet'] 
tweets_df.head()

,tweet
1175358310087892992,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1175416117793349632,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
1175450108898565888,@KanaanRema مبين من كلامه خليجي
1175471073770573824,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
1175496913145217024,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺


In [10]:
tweet_dialect_df = pd.merge(tweets_df, df, left_index=True ,right_on ='id')[['tweet','dialect']]
tweet_dialect_df.head()

,tweet,dialect
0,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .,IQ
1,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...,IQ
2,@KanaanRema مبين من كلامه خليجي,IQ
3,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐,IQ
4,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺,IQ


Ids have no benefit to be in the df, So I disposed of it

In [11]:
tweet_dialect_df.to_csv('tweets_dialect_dataset.csv' ,index=False)